In [42]:
# !pip install crewai crewai-tools langchain-community langchain-openai duckduckgo-search yfinance

In [1]:
# these three lines swap the stdlib sqlite3 lib with the pysqlite3 package
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# https://github.com/crewAIInc/crewAI/issues/2881#issuecomment-2904332609
# from rich.console import Console
# import sys
# import builtins

# # Create a safe console for Jupyter (no ANSI, no recursion)
# console = Console(force_terminal=False, file=sys.__stdout__)

# # Optional: make all `print()` use the safe console
# builtins.print = console.print

In [ ]:
# imports
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from crewai_tools import CSVSearchTool # RAG for CSV
import os
from datetime import datetime, timedelta
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = ""
llm = ChatOpenAI(model="gpt-4.1-mini")

In [4]:
# Search tool
from langchain_community.tools import DuckDuckGoSearchResults

searchTool = DuckDuckGoSearchResults(backend='api', num_results=10)

@tool
def search(query: str):
    """Searches information using DuckDuckGo."""
    return searchTool.invoke(query)

In [5]:
videoScriptwriter = Agent(
    role="Video Scriptwriter specialized in YouTube videos",
    goal="""Create a detailed, engaging, and structured script tailored for a full-length YouTube content, including hooks, storytelling, transitions, and calls to action.""",
    backstory="""You're a highly experienced in writing scripts for gaming YouTube videos.
        You also have a large experience in overall content creation.
        You are a storytelling specialist when it comes to gaming videos.
        You always search for new gaming content to make sure your gaming knowledge is up-to-date, always considering the source of the searched content.""",
    llm=llm,
    max_iter=5,
    allow_delegation=False, # can't delegate his task to another agent
    verbose=False,
    memory=True,
    tools=[search]
)
writeYoutubeVideoScript = Task(
    description="""Write a detailed, engaging, and structure script tailored for a full-length YouTube video script for {query}.
    Include hooks, storytelling, transitions, and calls to action.
    Make sure your knowledge regarding {query} is up-to-date by searching it from trusted sources such as official game websites, major gaming news outlets (e.g., IGN, Kotaku), or well-known community wikis.""",
    expected_output="""A well-written, structured, detailed YouTube video script for the proposed theme.""",
    agent=videoScriptwriter
)

# suggestion from GPT - not doing it for now
# expected_output="""A structured script using the following format:

# - Title: ...
# - Hook (0:00–0:30): ...
# - Intro (0:30–1:00): ...
# - Main Storyline (1:00–X): ...
# - Climax / Twist: ...
# - CTA (Call to Action): ...
# - Outro: ...

# The script should have timestamps, speaker tone suggestions (if relevant), and transitions clearly marked."""

In [6]:
thumbnailCreator = Agent(
    role="Thumbnail creator specialist",
    goal="""Create thumbnails inspired in a given YouTube video content, remarking attractive visual elements.""",
    backstory="""You are an expert in creating thumbnails for a variety of YouTube videos, always taking into account the video script and content.
    You create impressive thumbnails using visual elements that sparks curiosity in the viewers.""",
    llm=llm,
    max_iter=5,
    allow_delegation=True,
    verbose=False,
    memory=True,
    tools=[search]
)
createThumbnails = Task(
    description="""Create three variations of thumbnails for the YouTube video script using its content as an inspiration.
    If the video script lacks clarity or inspiration for thumbnail creation, you may delegate the task back to the original `videoScriptwriter` agent to revise or enrich the script content.""",
    expected_output="""Three thumbnail concepts with the following format:
    - Suggested title text (short, 3 to 6 words)
    - Description of imagery and layout (what's shown, colors, expressions, etc.)
    - Emotions or curiosity triggers used (e.g., surprise, suspense, humor)
    - Optional: callouts or design notes (e.g., “add red arrow pointing to treasure chest”)""",
    agent=thumbnailCreator,
    context=[writeYoutubeVideoScript]
)

In [15]:
videoScriptReviewer = Agent(
    role="Senior video script reviewer",
    goal="""Review the video script to ensure it is well-structured, has detailed content, is ready to be recorded, and has thumbnail options that matches the video content.""",
    backstory="""You are an unbelievable video script reviewer that understands what a high-quality YouTube video should have, and how it should be structured.
    "You always know when a thumbnail is appropriate for the video content, and if it will make the audience curious.""",
    llm=llm,
    max_iter=5,
    allow_delegation=True,
    verbose=False,
    memory=True
)
review = Task(
    description="""Review the YouTube video script and the associated thumbnails to write the final YouTube video script.
    Include the three thumbnail options, and write a brief summary on why each one should be used.
    If you think the video script and/or the thumbnails aren't good enough, you can delegate back to the video script writer and/or the thumbnail creator so they work again on their task.
    If a task is delegated, provide a specific comment explaining *what must be improved* (e.g., "script lacks a strong hook," or "thumbnail doesn’t match climax scene").""",
    expected_output="""A final, polished YouTube video script and three thumbnail options, with justifications for each.
        Output format:
        ---
        SCRIPT:
        <final video script content>

        THUMBNAILS:
        1. Thumbnail Concept 1
        - <detailed description of the thumbnail concept 1>
        - Justification: ...
        2. Thumbnail Concept 2
        - <detailed description of the thumbnail concept 1>
        - Justification: ...
        3. Thumbnail Concept 3
        - <detailed description of the thumbnail concept 1>
        - Justification: ...
   """,
    agent=videoScriptReviewer,
    context=[writeYoutubeVideoScript, createThumbnails]
)

In [16]:
crew = Crew(
    agents=[videoScriptwriter, thumbnailCreator, videoScriptReviewer],
    tasks=[writeYoutubeVideoScript, createThumbnails, review],
    verbose=False,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    manager_llm=llm
)

In [17]:
results = crew.kickoff(inputs={'query': 'Top 5 best games for 2024'})

In [18]:
from IPython.display import Markdown
Markdown(results.raw)

---
SCRIPT:
[Opening Scene: Energetic background music with dynamic gaming visuals]

Host (excited): "What’s up, gamers! Welcome back to the channel where we dive deep into the world of gaming greatness. If you’re ready to discover the absolute best games rocking 2024, you’ve landed in the right place! Today, we’re counting down the top 5 games that are either already out or dropping soon — and trust me, these titles will blow your mind. So strap in, grab your controller, and let’s jump right into the action!"

[Transition with game-themed animation]

---

#5: Alan Wake 2 (Released early 2024 on PC and consoles)  
Host: "Starting off strong at number five, we have *Alan Wake 2*, the highly anticipated sequel to the cult classic psychological thriller. Picture this: it’s dark, stormy, and you’re trapped in a world where reality twists and shadows come alive. Remedy Entertainment has taken everything fans loved about the first game and dialed up the horror and mystery to eleven. From eerie forests to mind-bending storylines, this game isn’t just about scares — it’s about storytelling that grips you like a vice. Released earlier this year, *Alan Wake 2* has been praised for its cinematic feel and intense atmosphere. If you’re into narrative-driven games that keep you on the edge of your seat, this one’s a must-play."

[Visuals: Clips of Alan Wake 2 gameplay and cinematic scenes]

---

#4: Starfield (Released mid-2024 on PC and Xbox)  
Host: "At number four, we’re blasting off into the cosmos with *Starfield*, Bethesda’s epic new space RPG. Imagine crafting your own destiny across a vast, unexplored galaxy — piloting starships, building bases, and engaging with alien civilizations. This game redefines open-world exploration with a sci-fi twist, offering a staggering amount of freedom and depth. The storytelling? Absolutely next-level. Bethesda’s knack for immersive worlds shines through every corner of *Starfield*. Whether you’re a fan of deep lore or just love the thrill of discovery, this game invites you to write your own story among the stars."

[Visuals: Starfield space exploration, ship battles, character interactions]

---

#3: The Legend of Zelda: Tears of the Kingdom (Released early 2024 on Nintendo Switch)  
Host: "Sliding into number three, Nintendo’s masterpiece *The Legend of Zelda: Tears of the Kingdom* has taken the gaming community by storm. Expanding on the revolutionary gameplay of *Breath of the Wild*, this sequel invites players back to Hyrule with new mechanics, breathtaking environments, and a story that’s as rich as it is mysterious. From soaring through the skies on your custom glider to solving mind-bending puzzles, this game blends exploration and adventure like no other. It’s a perfect example of how a beloved franchise can evolve while keeping its heart intact."

[Visuals: Zelda gameplay footage, epic landscapes, puzzle solving]

---

#2: Final Fantasy VII Rebirth (Coming late 2024 on PlayStation 5)  
Host: "Coming in hot at number two, *Final Fantasy VII Rebirth* continues the legendary saga of Cloud and his friends with stunning visuals and an emotionally charged story. This follow-up to the *Final Fantasy VII Remake* expands the world and deepens the narrative, delivering jaw-dropping moments that fans have been waiting for. Whether you’re a longtime fan or new to the series, the game’s rich character development and innovative combat system will pull you right into its captivating universe. Prepare for drama, action, and unforgettable music that stays with you long after the credits roll."

[Visuals: FFVII Rebirth cinematic scenes and combat gameplay]

---

#1: Diablo IV (Released mid-2024 on PC and consoles)  
Host: "And finally, the crown jewel at number one — *Diablo IV*! Blizzard’s dark fantasy RPG returns with a vengeance, delivering a gritty, immersive experience that has fans buzzing worldwide. With its vast open world, deep customization, and brutal combat, *Diablo IV* is the perfect blend of classic hack-and-slash and modern RPG mechanics. The story plunges you into a world plagued by ancient evils, where every decision matters. Whether you’re playing solo or teaming up with friends, the thrill of hunting loot and conquering demons is unmatched. This is the game that’s redefining the genre in 2024."

[Visuals: Diablo IV gameplay, character customization, intense battles]

---

[Closing Scene: Host back on screen with upbeat music]

Host: "And there you have it — the top 5 best games for 2024 that you absolutely cannot miss! Whether you’re into spine-tingling horror, cosmic adventures, legendary quests, epic RPGs, or demon-slaying action, this year’s lineup has something epic for every gamer out there. What game are you most hyped for? Drop your thoughts in the comments below — I want to hear all about your favorites and what you’re most excited to play!"

Host (smiling): "If you enjoyed this countdown, don’t forget to smash that like button, hit subscribe, and ring the bell so you never miss out on the latest gaming news and reviews. Thanks for watching, and until next time, keep gaming and stay awesome!"

[End Screen: Subscribe button animation, links to other videos]

---

THUMBNAILS:

1. Thumbnail Concept 1  
- A dynamic collage featuring iconic visuals from all five games: Alan Wake 2’s dark forest with shadows, Starfield’s spaceship against a starry galaxy, Zelda soaring on her glider over Hyrule, Cloud from Final Fantasy VII Rebirth in a dramatic pose, and a fierce demon from Diablo IV. The background blends moody blues with fiery reds, representing horror and action. The host’s excited face appears in a corner holding a game controller.  
- Justification: This collage captures the breadth of the countdown and appeals to fans of multiple games, generating excitement and anticipation for the full video.

2. Thumbnail Concept 2  
- Split-screen "versus" style: Alan Wake 2’s shadowy figure with glowing eyes on one side, Diablo IV’s fierce demon with a fiery background on the other, separated by a bold "VS" and a question mark with sparks. The dark palette with red and blue highlights creates suspense and intensity.  
- Justification: The competitive layout stirs curiosity and debate, engaging viewers who want to see which game ranks higher or why these two are featured prominently, driving clicks through intrigue.

3. Thumbnail Concept 3  
- A dramatic montage showing Zelda flying over Hyrule, a starship from Starfield launching, and an intense Diablo IV battle, highlighted with gold and neon green accents. The title text "Epic Games You Can't Miss" is bold at the top, with a glowing “Must Play!” sticker at the bottom right and subtle sparkles enhancing the energetic vibe.  
- Justification: The vibrant colors and dynamic imagery create urgency and excitement, compelling viewers to watch now so they don’t miss out on these standout 2024 games.